In [ ]:
import spacy
import json
import os
from nltk.corpus import stopwords
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions

files = []
for movie_year in tuple(os.walk("../imdb/imdb_movie_jsons"))[0][1]:
    subdir = tuple(os.walk(f"../imdb/imdb_movie_jsons/{movie_year}"))
    for file_name in tuple(subdir)[0][-1]:
        # print(f"{subdir[0][0]}/{file_name}")
        with open(f"{subdir[0][0]}/{file_name}", 'r', encoding="utf-8") as f:
            files.append(json.load(f))

In [ ]:
files = []
subdir = tuple(os.walk("../imdb/imdb_movie_jsons/1990"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [ ]:
corpus = []
my_pos = set(["ADJ", "ADV",])
nlp = spacy.load("en_core_web_sm")
# nlp.max_length = 50_000_000
for file in files[10::]:
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        string = nlp(review)
        clean = []
        for word in string:
            if word.pos_ in my_pos:
                clean.append(str(word.lemma_))
        corpus.append(" ".join(clean))
    # corpus = clean
    break

In [ ]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations

In [ ]:
with open("../stopwords.json", "r") as stops:
    stopwords = json.load(stops)
def clean(text: str):
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)
    text = " ".join([word for word in text.split() if word not in stopwords])

    return text.strip()

In [ ]:
proc_data = [clean(string.split()) for string in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [ ]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict)
vector = model[input_corpus]

In [ ]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f
    # break

In [ ]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(d.keys())

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)